## How can I leverage State-of-the-Art Natural Language Models with only one line of code ?

## Table of Content

[1. Sentence Classification - Sentiment Analysis](#1.-Sentence-Classification---Sentiment-Analysis)
[2. Token Classification - Named Entity Recognition](#2.-Token-Classification---Named-Entity-Recognition)
[3. Question Answering](#3.-Question-Answering)
[4. Text Generation - Mask Filling](#4.-Text-Generation---Mask-Filling)
[5. Projection - Features Extraction](#5.-Projection---Features-extraction) 


Newly introduced in transformers v2.3.0, **pipelines** provides a high-level, easy-to-use,
API for doing inference over a variety of downstream-tasks, including: 

- Sentence Classification (Sentiment Analysis): Indicate if the overall sentence is either positive or negative. _(Binary Classification task or Logitic Regression task)_
- Token Classification (Named Entity Recognition, Part-of-Speech tagging): For each sub-entities _(**tokens**)_ in the input, assign them a label _(Classification task)_.
- Question-Answering: Provided a tuple (question, context) the model should find the span of text in **content** answering the **question**.
- Mask-Filling: Generates the next word provided an initial **context**.
- Feature Extraction: Maps the input to a higher, multi-dimensional space learned from the data.

Pipelines encapsulate the overall process of every NLP process:
 
 1. Tokenization: Split the initial input into multiple sub-entities with ... properties (i.e. tokens).
 2. Inference: Maps every tokens into a more meaningful representation. 
 3. Decoding: Use the above representation to generate and/or extract the final output for the underlying task.

The overall API is exposed to the end-user through the `pipeline()` method with the following 
structure:

```python
from transformers import pipeline

# Using default model and tokenizer for the task
pipeline("<task-name>")

# Using a user-specified model
pipeline("<task-name>", model="<model_name>")

# Using custom model/tokenizer as str
pipeline('<task-name>', model='<model name>', tokenizer='<tokenizer_name>')
```

In [29]:
import numpy as np
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from transformers import pipeline

SyntaxError: from __future__ imports must occur at the beginning of the file (<ipython-input-29-c3a037bd4c55>, line 5)

## 1. Sentence Classification - Sentiment Analysis

In [6]:
nlp_sentence_classif = pipeline('sentiment-analysis')
nlp_sentence_classif(['Such a nice weather outside !', 'This movie was kind of boring.'])

[{'label': 'POSITIVE', 'score': 0.800251},
 {'label': 'NEGATIVE', 'score': 1.2489903}]

## 2. Token Classification - Named Entity Recognition

In [16]:
nlp_token_class = pipeline('ner')
nlp_token_class('Hugging Face is a French company based in New-York.')

[{'word': 'Hu', 'score': 0.9970937967300415, 'entity': 'I-ORG'},
 {'word': '##gging', 'score': 0.9345750212669373, 'entity': 'I-ORG'},
 {'word': 'Face', 'score': 0.9787060022354126, 'entity': 'I-ORG'},
 {'word': 'French', 'score': 0.9981995820999146, 'entity': 'I-MISC'},
 {'word': 'New', 'score': 0.9983047246932983, 'entity': 'I-LOC'},
 {'word': '-', 'score': 0.8913455009460449, 'entity': 'I-LOC'},
 {'word': 'York', 'score': 0.9979523420333862, 'entity': 'I-LOC'}]

## 3. Question Answering

In [18]:
nlp_qa = pipeline('question-answering')
nlp_qa(context='Hugging Face is a French company based in New-York.', question='Where is based Hugging Face ?')

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 2673.23it/s]


{'score': 0.9632966867654424, 'start': 42, 'end': 50, 'answer': 'New-York.'}

## 4. Text Generation - Mask Filling

In [20]:
nlp_fill = pipeline('fill-mask')
nlp_fill('Hugging Face is a French company based in <mask>')

[{'sequence': '<s> Hugging Face is a French company based in Paris</s>',
  'score': 0.25288480520248413,
  'token': 2201},
 {'sequence': '<s> Hugging Face is a French company based in Lyon</s>',
  'score': 0.07639515399932861,
  'token': 12790},
 {'sequence': '<s> Hugging Face is a French company based in Brussels</s>',
  'score': 0.055500105023384094,
  'token': 6497},
 {'sequence': '<s> Hugging Face is a French company based in Geneva</s>',
  'score': 0.04264815151691437,
  'token': 11559},
 {'sequence': '<s> Hugging Face is a French company based in France</s>',
  'score': 0.03868963569402695,
  'token': 1470}]

## 5. Projection - Features Extraction 

In [32]:
import numpy as np
nlp_features = pipeline('feature-extraction')
output = nlp_features('Hugging Face is a French company based in Paris')
np.array(output).shape   # (Samples, Tokens, Vector Size)


(1, 12, 768)

Alright ! Now you have a nice picture of what is possible through transformers' pipelines, and there is more
to come in future release. 

In the meantime, you can try the different pipelines with your own inputs

In [40]:
task = widgets.Dropdown(
    options=['sentiment-analysis', 'ner', 'fill_mask'],
    value='ner',
    description='Task:',
    disabled=False
)

input = widgets.Text(
    value='',
    placeholder='Enter something',
    description='Your input:',
    disabled=False
)

def forward(_):
    if len(input.value) > 0: 
        if task.value == 'ner':
            output = nlp_token_class(input.value)
        elif task.value == 'sentiment-analysis':
            output = nlp_sentence_classif(input.value)
        else:
            if input.value.find('<mask>') == -1:
                output = nlp_fill(input.value + ' <mask>')
            else:
                output = nlp_fill(input.value)                
        print(output)

input.on_submit(forward)
display(task, input)

Dropdown(description='Task:', index=1, options=('sentiment-analysis', 'ner', 'fill_mask'), value='ner')

Text(value='', description='Your input:', placeholder='Enter something')

[{'word': 'Paris', 'score': 0.9991844296455383, 'entity': 'I-LOC'}]
[{'sequence': '<s> I\'m from Paris."</s>', 'score': 0.224044069647789, 'token': 72}, {'sequence': "<s> I'm from Paris.)</s>", 'score': 0.16959427297115326, 'token': 1592}, {'sequence': "<s> I'm from Paris.]</s>", 'score': 0.10994981974363327, 'token': 21838}, {'sequence': '<s> I\'m from Paris!"</s>', 'score': 0.0706234946846962, 'token': 2901}, {'sequence': "<s> I'm from Paris.</s>", 'score': 0.0698278620839119, 'token': 4}]
[{'sequence': "<s> I'm from Paris and London</s>", 'score': 0.12238534539937973, 'token': 928}, {'sequence': "<s> I'm from Paris and Brussels</s>", 'score': 0.07107886672019958, 'token': 6497}, {'sequence': "<s> I'm from Paris and Belgium</s>", 'score': 0.040912602096796036, 'token': 7320}, {'sequence': "<s> I'm from Paris and Berlin</s>", 'score': 0.039884064346551895, 'token': 5459}, {'sequence': "<s> I'm from Paris and Melbourne</s>", 'score': 0.038133684545755386, 'token': 5703}]
[{'sequence': 

In [43]:
context = widgets.Textarea(
    value='Einstein is famous for the general theory of relativity',
    placeholder='Enter something',
    description='Context:',
    disabled=False
)

query = widgets.Text(
    value='Why is Einstein famous for ?',
    placeholder='Enter something',
    description='Question:',
    disabled=False
)

def forward(_):
    if len(context.value) > 0 and len(query.value) > 0: 
        output = nlp_qa(question=query.value, context=context.value)            
        print(output)

query.on_submit(forward)
display(context, query)

Textarea(value='Einstein is famous for the general theory of relativity', description='Context:', placeholder=…

Text(value='Why is Einstein famous for ?', description='Question:', placeholder='Enter something')

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 1919.59it/s]


{'score': 0.40340670623875496, 'start': 27, 'end': 54, 'answer': 'general theory of relativity'}
